In [ ]:
import requests
import pandas as pd
import numpy as np
import nbconvert
import datetime
import plotly.graph_objects as go
import warnings; warnings.simplefilter('ignore')
from IPython.display import display, Markdown, Image, SVG
import re
import scrapbook as sb

from bmondata import Server
from dateutil.relativedelta import relativedelta

import bmondata

In [ ]:
# Parameters to be changed/exported using Papermill or Scrapbook
building_id = 2
server_web_address = 'https://bms.ahfc.us'

In [ ]:
# Set false as default, then change later if we need to hide this page
hide = False

# 'Glue' down variables for scrapbook to export later
sb.glue('sort_order', 5)
sb.glue('title', 'Indoor Temp')

server = Server(server_web_address)

building_df = server.buildings(building_id)
current_building_name = building_df[0]['title']

indoor_temps = bmondata.csnl_to_list(server.buildings(building_id)[0]['indoor_temps'])

if len(indoor_temps) == 0:
    error_message = 'This building does not appear to have any indoor temperature data. Either there is no data source or it has not been properly configured.'
    raise RuntimeError(error_message)

# Check to see how long ago the most recent data was posted to BMON. If there has been no data for at least a week, 
# then report an error message and stop the analysis
most_recent_data = server.sensor_readings(indoor_temps).index.values[-1]
nanos_since_data_posted = np.datetime64('now') - most_recent_data
days_since_data_posted = nanos_since_data_posted.astype('timedelta64[D]') / np.timedelta64(1, 'D')

if days_since_data_posted > 7:
    sensor_error_message = 'The temperature sensor for this building has not posted any data for at least 7 days. Please check the sensor to ensure it is running and properly configured.'
    raise RuntimeError(sensor_error_message)

if server.sensor_readings(indoor_temps,
                                start_ts = (datetime.datetime.now() - relativedelta(weeks=1)),
                                end_ts = datetime.datetime.now(),
                                               averaging='15min').empty:
    if server.sensor_readings(indoor_temps,
                                start_ts = (datetime.datetime.now() - relativedelta(weeks=1)),
                                end_ts = datetime.datetime.now(),
                                               averaging='1H').empty:
        indoor_temps_current_week = server.sensor_readings(indoor_temps,
                                start_ts = (datetime.datetime.now() - relativedelta(weeks=1)),
                                end_ts = datetime.datetime.now(),
                                               averaging='2H')
    else: 
        indoor_temps_current_week = server.sensor_readings(indoor_temps,
                                start_ts = (datetime.datetime.now() - relativedelta(weeks=1)),
                                end_ts = datetime.datetime.now(),
                                               averaging='1H')
else:
    indoor_temps_current_week = server.sensor_readings(indoor_temps,
                                    start_ts = (datetime.datetime.now() - relativedelta(weeks=1)),
                                    end_ts = datetime.datetime.now(),
                                                   averaging='15min')

diverging_hue_list = ['#a6cee3','#1f78b4','#b2df8a',
                      '#33a02c','#fb9a99','#e31a1c',
                      '#fdbf6f','#ff7f00','#cab2d6',
                      '#6a3d9a']

title_md = '''# Indoor Temperatures for {} building'''
title_md = title_md.format(current_building_name)

#################################################################

Markdown(title_md)

In [ ]:
print(current_building_name)

In [ ]:
# Get schedule and timezone from the API data
building_schedule = building_df[0]['schedule']
building_timezone = building_df[0]['timezone']

# Create a schedule object using Ian's library
schedule_object = bmondata.Schedule(building_schedule, building_timezone)

#######################################################################

if building_schedule == '' or schedule_object is None:
    md = '''#### <font color='red'>This building does not appear to have any defined occupied schedule. Please define the occupied hours through the administrator interface.</font>'''
else:
    
    md = '## Grey areas indicate that the building is unoccupied; to save energy and money, ensure that the heating control system is programmed to reduce the building temperature during these periods.' 
    
    # Set the start and end times of the graph
    graph_start_date = datetime.datetime.now() - relativedelta(weeks=1)
    graph_end_date = datetime.datetime.now()

    # Use the schedule object to create a list of tuples with the occupied start and end times falling within the graph range
    list_of_occupied_timestamps = schedule_object.occupied_periods(datetime.datetime.timestamp(graph_start_date), datetime.datetime.timestamp(graph_end_date))

    # Loop through the list of occupied timestamps and convert them to datetimes
    start_time_list = []
    end_time_list = []

    for i in np.arange(0, len(list_of_occupied_timestamps)):
        start_datetime = datetime.datetime.fromtimestamp(list_of_occupied_timestamps[i][0])
        start_time_list.append(start_datetime)
        end_datetime = datetime.datetime.fromtimestamp(list_of_occupied_timestamps[i][1])
        end_time_list.append(end_datetime)

    # Create variables to indicate the first occupied start time and the last occupied end time
    occ_start = start_time_list[0]
    occ_end = end_time_list[-1]
    occ_day_list = np.arange(occ_start.weekday(), occ_end.weekday()+1)
    
    min_value = indoor_temps_current_week.min().min()
    max_value = indoor_temps_current_week.max().max()
    
    shape_list = []
    shape_dict = {'type':'rect',
                  'fillcolor':'#bdbdbd',
                  'opacity':0.35,
                  'line': {'width':1},
                  'layer':'below',
                  'y0':min_value,
                  'y1':max_value
                 }
    
    if graph_start_date < start_time_list[0]:
        shape_dict.update(x0=graph_start_date)
        shape_dict.update(x1=start_time_list[0])
        shape_list.append(shape_dict.copy())
        
    for i in np.arange(len(start_time_list)):
        if start_time_list[-1] != start_time_list[i]:
            shape_dict.update(x0=end_time_list[i])
            shape_dict.update(x1=start_time_list[i+1])
            shape_list.append(shape_dict.copy())
            
    if graph_end_date > end_time_list[-1]:
        shape_dict.update(x0=end_time_list[-1])
        shape_dict.update(x1=graph_end_date)
        shape_list.append(shape_dict.copy())
        
    counter = 1

    data = []
    for temp_sensor in indoor_temps:
        sensor_title = server.sensors(temp_sensor)[0]['title']
        
        temp_sensor = go.Scatter(x=indoor_temps_current_week.index,
                                 y=indoor_temps_current_week[temp_sensor],
                                 name=sensor_title,
                                 line=dict(color=diverging_hue_list[(counter-1)]))
        counter += 1
        data.append(temp_sensor)

    layout = dict(title='Unoccupied Indoor Temperatures',
                  xaxis=dict(title='Date and Time'),
                  yaxis=dict(title='Indoor Temp (F)'),
                  shapes = shape_list,
                  showlegend = True
                 )


    fig = go.Figure(dict(data=data, layout=layout))
    fig.show()

Markdown(md)

In [ ]:
min_value = indoor_temps_current_week.min().min()
max_value = indoor_temps_current_week.max().max()

counter = 1

data = []
for temp_sensor in indoor_temps:
    sensor_title = server.sensors(temp_sensor)[0]['title']
    
    temp_sensor = go.Scatter(x=indoor_temps_current_week.index,
                             y=indoor_temps_current_week[temp_sensor],
                             ## to change temp_sensor to sensor_title ##
                             name=sensor_title,
                             line=dict(color=diverging_hue_list[(counter-1)]))
    counter += 1
    data.append(temp_sensor)
    
middle_x = ((indoor_temps_current_week.index.max() - indoor_temps_current_week.index.min()) / 2) + indoor_temps_current_week.index.min()
middle_y = (max_value - 74)/2 + 74
# Add in explanatory text
text1 = go.Scatter(x = [middle_x],
                   y = [middle_y],
                   mode='text',
                   text=['<b>Indoor temperatures in this range are higher than typical; <br /> consider lowering the thermostat to save energy and money'],
                    textposition='middle center',
                   showlegend=False,
                   textfont=dict(color="black",size=22)
                  )


data.append(text1)

if max_value > 74:
    temp_bands = [
          {'type':'rect',
                        'x0':indoor_temps_current_week.index.min(),
                        'y0':74,
                        'x1':indoor_temps_current_week.index.max(),
                        'y1':max_value,
                        'fillcolor':'#d7191c',
                        'opacity':0.35,
                        'line': {
                            'width':1,
                        }
                        }
         ]
else: 
    temp_bands = []


layout = dict(title='Current Week Indoor Temperatures: Excess Heating',
              xaxis=dict(title='Date and Time'),
              yaxis=dict(title='Indoor Temp (F)'),
              shapes = temp_bands,
              showlegend = True
             )


fig = go.Figure(dict(data=data, layout=layout))

indoor_temps_current_week = indoor_temps_current_week.reset_index()

indoor_temps_current_week['hour'] = indoor_temps_current_week['index'].apply(lambda x: x.hour)

indoor_temps_current_week['daytime'] = np.where((indoor_temps_current_week.hour >= 8) & (indoor_temps_current_week.hour <= 17),
                                               1, 0)

daytime_temps = indoor_temps_current_week.query("daytime == 1")
daytime_temps = daytime_temps.drop(columns=['hour', 'daytime', 'index'])

####################################################################

if daytime_temps.mean().mean() > 45:
    fig.show()